In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchmetrics import MeanAbsolutePercentageError
import torch.nn as nn
import torch.optim as optim
import tqdm

In [4]:
# Pobieranie i dzielenie danych
dataset = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2, random_state=123)

# Standaryzacja danych
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Konwersja danych do tensorów PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [5]:
# Funkcja do trenowania modelu
def train_model(layers, activations, dropout_rate, epochs, learning_rate):
    model_layers = []
    input_dim = 8

    for layer_size, activation in zip(layers, activations):
        model_layers.append(nn.Linear(input_dim, layer_size))
        if activation == "relu":
            model_layers.append(nn.ReLU())
        elif activation == "leakyrelu":
            model_layers.append(nn.LeakyReLU())
        elif activation == "tanh":
            model_layers.append(nn.Tanh())
        if dropout_rate > 0:
            model_layers.append(nn.Dropout(dropout_rate))
        input_dim = layer_size

    model_layers.append(nn.Linear(input_dim, 1))
    model = nn.Sequential(*model_layers)

    loss_fn = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    mape = MeanAbsolutePercentageError()

    loss_list = np.zeros((epochs,))
    mse_list = np.zeros((epochs,))
    mape_list = np.zeros((epochs,))

    for epoch in tqdm.trange(epochs):
        model.train()
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        loss_list[epoch] = loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test)
            mse_list[epoch] = loss_fn(y_pred, y_test).item()
            mape_list[epoch] = mape(y_pred, y_test).item()

    return min(mape_list), model

In [7]:
# Definiowanie hiperparametrów do przetestowania
layer_configs = [
    [128, 128, 64, 32],
    [256, 128, 64],
    [128, 64, 32],
]

activation_configs = [
    ["relu", "relu", "relu", "relu"],
    ["leakyrelu", "leakyrelu", "leakyrelu", "leakyrelu"],
    ["tanh", "tanh", "tanh", "tanh"],
]

dropout_rates = [0.2, 0.3, 0.4]
epoch_numbers = [1000, 2000]
learning_rates = [0.0001, 0.0005]

In [8]:
# Przeszukiwanie przestrzeni hiperparametrów
best_mape = float('inf')
best_config = None
best_model = None

for layers in layer_configs:
    for activations in activation_configs:
        for dropout_rate in dropout_rates:
            for epochs in epoch_numbers:
                for lr in learning_rates:
                    print(f"Training with layers={layers}, activations={activations}, dropout={dropout_rate}, epochs={epochs}, lr={lr}")
                    mape, model = train_model(layers, activations, dropout_rate, epochs, lr)
                    print(f"MAPE: {mape}")
                    if mape < best_mape:
                        best_mape = mape
                        best_config = (layers, activations, dropout_rate, epochs, lr)
                        best_model = model

Training with layers=[128, 128, 64, 32], activations=['relu', 'relu', 'relu', 'relu'], dropout=0.2, epochs=1000, lr=0.0001


100%|██████████| 1000/1000 [00:37<00:00, 26.71it/s]


MAPE: 0.22528579831123352
Training with layers=[128, 128, 64, 32], activations=['relu', 'relu', 'relu', 'relu'], dropout=0.2, epochs=1000, lr=0.0005


100%|██████████| 1000/1000 [00:39<00:00, 25.64it/s]


MAPE: 0.18629492819309235
Training with layers=[128, 128, 64, 32], activations=['relu', 'relu', 'relu', 'relu'], dropout=0.2, epochs=2000, lr=0.0001


...

MAPE: 0.2597244381904602
Training with layers=[128, 64, 32], activations=['tanh', 'tanh', 'tanh', 'tanh'], dropout=0.4, epochs=2000, lr=0.0001


100%|██████████| 2000/2000 [00:44<00:00, 45.14it/s]


MAPE: 0.30846917629241943
Training with layers=[128, 64, 32], activations=['tanh', 'tanh', 'tanh', 'tanh'], dropout=0.4, epochs=2000, lr=0.0005


100%|██████████| 2000/2000 [00:44<00:00, 45.29it/s]

MAPE: 0.247308149933815


In [ ]:
# Best MAPE: 0.17220439016819
# Best configuration: Layers=[256, 128, 64], Activations=['relu', 'relu', 'relu', 'relu'], Dropout=0.2, Epochs=2000, Learning rate=0.0005

In [9]:
print(f"Best MAPE: {best_mape}")
print(f"Best configuration: Layers={best_config[0]}, Activations={best_config[1]}, Dropout={best_config[2]}, Epochs={best_config[3]}, Learning rate={best_config[4]}")

plt.plot(mse_list, label="MSE")
plt.plot(mape_list, label="MAPE")
plt.ylabel("validation MSE and MAPE")
plt.xlabel("epochs")
plt.legend()
plt.show()

Best MAPE: 0.17220439016819
Best configuration: Layers=[256, 128, 64], Activations=['relu', 'relu', 'relu', 'relu'], Dropout=0.2, Epochs=2000, Learning rate=0.0005


NameError: name 'mse_list' is not defined

In [ ]:
# Funkcja do trenowania modelu
def train_model_v2(layers, activations, epochs, learning_rate):
    model_layers = []
    input_dim = 8

    for layer_size, activation in zip(layers, activations):
        model_layers.append(nn.Linear(input_dim, layer_size))
        if activation == "relu":
            model_layers.append(nn.ReLU())
        elif activation == "leakyrelu":
            model_layers.append(nn.LeakyReLU())
        elif activation == "tanh":
            model_layers.append(nn.Tanh())
        input_dim = layer_size

    model_layers.append(nn.Linear(input_dim, 1))
    model = nn.Sequential(*model_layers)

    loss_fn = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    mape = MeanAbsolutePercentageError()

    loss_list = np.zeros((epochs,))
    mse_list = np.zeros((epochs,))
    mape_list = np.zeros((epochs,))

    for epoch in tqdm.trange(epochs):
        model.train()
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        loss_list[epoch] = loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test)
            mse_list[epoch] = loss_fn(y_pred, y_test).item()
            mape_list[epoch] = mape(y_pred, y_test).item()

    return min(mape_list), model

In [ ]:
# Przeszukiwanie przestrzeni hiperparametrów
best_mape = float('inf')
best_config = None
best_model = None

for layers in layer_configs:
    for activations in activation_configs:
        for epochs in epoch_numbers:
            for lr in learning_rates:
                print(f"Training with layers={layers}, activations={activations}, epochs={epochs}, lr={lr}")
                mape, model = train_model(layers, activations, epochs, lr)
                print(f"MAPE: {mape}")
                if mape < best_mape:
                    best_mape = mape
                    best_config = (layers, activations, epochs, lr)
                    best_model = model

print(f"Best MAPE: {best_mape}")
print(f"Best configuration: Layers={best_config[0]}, Activations={best_config[1]}, Epochs={best_config[2]}, Learning rate={best_config[3]}")

plt.plot(mse_list, label="MSE")
plt.plot(mape_list, label="MAPE")
plt.ylabel("validation MSE and MAPE")
plt.xlabel("epochs")
plt.legend()
plt.show()